## DOCUMENT SIMILARITY SEARCH USING PANDAS ON SINGLE MACHINE

In [2]:
import pandas as pd

In [4]:
# Loading dataset
file_path = "./dataset/corpus.jsonl"

chunk_size = 100000

chunks = []

for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    chunks.append(chunk)

df = pd.concat(chunks)
df.head(10)

,_id,title,text,metadata
0,12,Anarchism,Anarchism is a political philosophy that advoc...,{'url': 'https://en.wikipedia.org/wiki?curid=12'}
1,25,Autism,Autism is a neurodevelopmental disorder charac...,{'url': 'https://en.wikipedia.org/wiki?curid=25'}
2,39,Albedo,Albedo ( ) is a measure for reflectance or opt...,{'url': 'https://en.wikipedia.org/wiki?curid=39'}
3,290,A,"A (named , plural ""As"", ""A's"", ""a""s, ""a's"" or ...",{'url': 'https://en.wikipedia.org/wiki?curid=2...
4,303,Alabama,Alabama ( ) is a state in the southeastern reg...,{'url': 'https://en.wikipedia.org/wiki?curid=3...
5,305,Achilles,"In Greek mythology, Achilles ( , ; ] ) was a G...",{'url': 'https://en.wikipedia.org/wiki?curid=3...
6,307,Abraham Lincoln,"Abraham Lincoln ( ; February 12, 1809 – April ...",{'url': 'https://en.wikipedia.org/wiki?curid=3...
7,308,Aristotle,"Aristotle ( ; Greek: Ἀριστοτέλης , , ""Aristoté...",{'url': 'https://en.wikipedia.org/wiki?curid=3...
8,309,An American in Paris,An American in Paris is a jazz-influenced orch...,{'url': 'https://en.wikipedia.org/wiki?curid=3...
9,316,Academy Award for Best Production Design,The Academy Award for Best Production Design r...,{'url': 'https://en.wikipedia.org/wiki?curid=3...


In [5]:
print("Dataset size: ", len(df))

Dataset size:  5233329
